In [ ]:
### PREAMBLE
# "Unsupervised learning&#58; clustering and dimensionality reduction"
# unsupervised.svg

import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline
%config InlineBackend.figure_format = 'svg'

# Aprendizaje no supervisado

Hasta ahora, hemos presentado los métodos de aprendizaje automático en gran parte en el contexto del aprendizaje supervisado. Esto significa que hablamos de que los conjuntos de datos de aprendizaje automático tienen entradas x , salidas y , y el objetivo de un algoritmo de aprendizaje automático es aprender a predecir y a partir de x. El aprendizaje supervisado domina tradicionalmente la mayoría de las aplicaciones prácticas de la ciencia de los datos pero esta clase discutiremos los enfoques que difieren de esto. En particular, consideraremos la configuración del **aprendizaje no supervisado**. Aquí, no se nos dan los pares correspondientes de entrada/salida, sino que sólo se nos dan las entradas x . Por supuesto, esto plantea la pregunta obvia: **si no se nos da una salida objetivo**, ¿qué se supone que debemos predecir exactamente? Y de hecho, la tarea de aprendizaje sin supervisión es ambigua en algún nivel. Sin embargo, la filosofía general del **aprendizaje no supervisado es que queremos descubrir algún tipo de estructura en los datos**. Los diferentes métodos de aprendizaje no supervisado funcionan de maneras muy diferentes, y descubren muy diferentes tipos de estructura, pero todos tienen este elemento similar.



Vamos a centrarnos aquí en definir primero el aprendizaje no supervisado de manera genérica, utilizando las mismas nociones de funciones de hipótesis, funciones de pérdida y procedimientos de optimización que utilizamos para el aprendizaje supervisado.  Resultará que a pesar del contexto muy diferente, prácticamente todos los métodos de aprendizaje no supervisado también pueden ser vistos de esta manera.  Pero naturalmente, la forma en que definimos las hipótesis y las funciones de pérdida tendrá que cambiar para tener en cuenta el hecho de que ya no tenemos un objetivo bien definido al que tratamos de ajustarnos.  Después de un aprendizaje no supervisado definido en este contexto genérico, describiremos dos algoritmos particulares de aprendizaje no supervisado e ilustraremos cómo encajan en este marco: el algoritmo de agrupación de k-medios y el algoritmo de análisis de componentes principales (PCA).


## Construir un algoritmo de aprendizaje no supervisado

Recordemos de nuestras presentaciones sobre el aprendizaje supervisado que los tres aspectos de un algoritmo de aprendizaje supervisado son: 

1.   Una función de hipótesis
2.   Una función de pérdida
3.   Un método para minimizar la pérdida media sobre los datos de entrenamiento.

## K-means clustering

Como nuestro primer ejemplo de un algoritmo de aprendizaje supervisado, vamos a considerar el algoritmo de agrupación de k-medias, para ver cómo encaja en el marco anterior. Antes de presentar el marco formal, sin embargo, vamos a considerar el cluster de *k-means* desde un punto de vista más visual/geométrico, como una forma de agrupar puntos de datos.

Consideremos el siguiente conjunto de 100 puntos de datos.


Incluso si no miraras el código que generó estos puntos, debería ser bastante evidente que hay dos grupos de puntos diferentes aquí.   El objetivo del algoritmo de k-means es encontrar un conjunto de $k$ "centros" (es decir, puntos  $\mu^{(i)} \in \mathbb{R}^n$) de tal manera que cada punto de datos está cerca de al menos un centro.  Al hacerlo, también podemos asociar cada punto con su centro más cercano, y usar esto como una indicación de a qué cúmulo pertenece.  Veamos esto gráficamente para nuestro ejemplo anterior, usando el conocimiento de que generamos los datos para tener dos cúmulos explícitos.

### Definición formal del algoritmo K-means
---

Veamos cómo funciona esto en el entorno formal que hemos descrito anteriormente.

**Función de hipótesis** Primero discutamos la función de hipótesis.  Los parámetros $\theta$ de nuestra función de hipótesis sólo incluyen los centros mismos.

\begin{equation}
\theta = \{\mu^{(1)}, \ldots, \mu^{(k)}\}
\end{equation}

Por cada  $\mu^{(i)} \in \mathbb{R}^n$ (observe que, como será un patrón a partir de ahora, seguimos refiriéndonos a _todos_ los parámetros de nuestra hipótesis usando la notación $\theta$, pero cada vez más esto se referirá a una colección de parámetros en lugar de un solo vector de parámetros). Definida por estos parámetros, la función de hipótesis en sí misma, $h_\theta(x)$, sólo _sale del centro que está más cerca del punto $x$_.  Escrito formalmente

\begin{equation}
\DeclareMathOperator*{argmin}{argmin}
h_\theta(x) = \argmin_{\mu \in \{\mu^{(1)}, \ldots, \mu^{(k)}\} } \|\mu - x\|_2^2
\end{equation}

where the $\argmin$ operator returns the argument that minimizes the expression (as opposed to the $\min$ operator which just returns the minimum value), i.e., the expression just outputs whichever center $\mu^{(1)},\ldots,\mu^{(k)}$ is closest to $x$.

Donde el operador $\argmin$ devuelve el argumento que minimiza la expresión (a diferencia del operador $\min$ que sólo devuelve el valor mínimo), es decir, la expresión sólo produce el centro $\mu^{(1)},\ldots,\mu^{(k)}$ que esté más cerca de $x$.

**Loss function** La función de pérdida usada por _k-means_ es simplemente la pérdida al cuadrado que mencionamos antes.

\begin{equation}
\ell(h_\theta(x), x) = \|h_\theta(x) - x\|_2^2.
\end{equation}

Recordando que la función de hipótesis simplemente produce la media más cercana al punto dado $x$, esto equivale a decir que la pérdida para cualquier punto es simplemente la distancia al cuadrado entre el centro más cercano y ese punto, es decir,

\begin{equation}
\ell(h_\theta(x), x) = \min_{\mu \in \{\mu^{(1)}, \ldots, \mu^{(k)}\}} \|\mu - x\|_2^2.
\end{equation}

**Optimization** Por último, consideremos ahora que el problema de la optimización que resulta de la hipótesis y la pérdida arriba mencionadas,

\begin{equation}
minimize_\theta \;\; \frac{1}{m} \sum_{i=1}^m \min_{\mu \in \{\mu^{(1)}, \ldots, \mu^{(k)}\}} \|\mu - x^{(i)}\|_2^2.
\end{equation}

A diferencia de los ajustes que hemos visto antes en el aprendizaje supervisado, en este caso no es estándar optimizar simplemente esta pérdida usando el descenso de gradiente (aunque enfatizamos que esto _podría_ hacerse).  En su lugar, una estrategia común es "asignar" iterativamente cada punto a su centro más cercano (es decir, calcular el término que alcanza los $\min$ para cada punto de entrenamiento), luego actualizar cada centro para que sea la media de estos puntos asignados (esta asignación es precisamente la que minimiza la pérdida asumiendo que las asignaciones de los centros son fijas), y repetir este proceso hasta la convergencia.  Esto a veces se llama algoritmo de Lloyd, pero más típicamente se le llama simplemente "k-means", ya que es el método estándar para el entrenamiento de los modelos k-means.  Hay muchas maneras de asignar inicialmente los centros de cúmulos, pero una estrategia común es simplemente elegir $k$ de los puntos de datos al azar.  Formalmente, el algoritmo procede de la siguiente manera.

<hr/>
**Algorithm**: K-means

**Given:** Data set $x^{(i)}, i=1,\ldots,m$

**Initialize:**
    
* $\mu^{(j)} := \mbox{RandomChoice}(x^{(1:m)}), \; j=1,\ldots,k$

**Repeat until convergence:**

* Assign point clusters: $y^{(i)} := \argmin_j \|\mu^{(j)} - x^{(i)}\|_2^2, \; i=1,\ldots,m$
* Compute new centers: $\displaystyle \mu^{(j)} := \sum_{i=1}^m \frac{x^{(i)} \mathrm{1}\{y^{(i)} = j\}}{\mathrm{1}\{y^{(i)} = j\}}, \;\; j=1,\ldots,k$

<hr/>

### Análisis de k-means

Veamos cómo funciona esto en la práctica. La siguiente es una implementación razonablemente eficiente del algoritmo. El aspecto clave que lo hace eficiente es que calculamos las distancias entre todos los puntos y todos los centros usando operaciones de matriz (la misma estrategia se usó anteriormente cuando creamos las características del RBF en nuestro entorno de clasificación no lineal).  Para simplificar, aquí ejecutamos el algoritmo para un número fijo de iteraciones en lugar de comprobar la convergencia explícitamente.

La primera línea asigna aleatoriamente los centros a $k$ de los puntos de datos.  Luego repetimos para las iteraciones `max_iter`, cada vez:

1. Calculando la distancia entre todos los puntos y todos los cúmulos, `D`.
2. Computando la asignación de los clusters de cada punto, `y`.
3. Recalcular los centros para que sean el promedio de sus puntos.

Finalmente devolvemos estos cúmulos más la propia pérdida cuadrada.

Veamos cómo se ve esto en un ejemplo de tres grupos.  Nuestros datos sintéticos se generarán a partir de tres cúmulos diferentes.

También extenderemos el código anterior para este ejemplo en particular para trazar las medias del cúmulo en sus asignaciones en cada iteración del algoritmo.

Vamos a ejecutar esto en nuestro simple ajuste de tres grupos.  Aquí mostramos las primeras cuatro iteraciones del algoritmo, después de las cuales ya ha convergido.

Sin embargo, nótese que k-means es particularmente susceptible de quedarse atascado en el óptimo local.  Por ejemplo, con una inicialización aleatoria diferente, podemos obtener el siguiente comportamiento en su lugar.

### Seleccionando $k$

Una pregunta obvia que surge con este método es: ¿cómo elegimos los hiperparámetros para el algoritmo k-means, como el número de centros $k$?  Al igual que en el aprendizaje supervisado, no hay manera de evaluar directamente esto a partir de la pérdida de entrenamiento en sí; modulo la posibilidad de optima local, la pérdida debe seguir disminuyendo para un mayor número de centros (cuantos más centros, más cerca estará cualquier punto dado de ellos).  Pero, a diferencia del aprendizaje supervisado, no hay ni siquiera un buen análogo de la validación cruzada que podamos utilizar aquí: esta propiedad de menor pérdida se aplicará típicamente _también_ a un conjunto de validación.

Por esta razón, el proceso de selección del número de cúmulos es típicamente un arte impreciso, y es _muy_ difícil inferir algo sobre el número "real" de cúmulos en los datos a partir de la ejecución de k-means (de hecho, realmente nunca se debe intentar hacer esto).  Una estrategia común es más bien trazar la pérdida frente al número de cúmulos e intentar encontrar un punto que sea "suficientemente bueno" en términos de pérdida frente al número de cúmulos (es decir, en el que añadir cúmulos adicionales no ayude mucho).

Hagamos esto primero en nuestros ejemplos sintéticos.


### Ejemplo: Dígitos escritos a mano

Como ejemplo, consideremos la tarea de agrupar imágenes de dígitos manuscritos del conjunto de datos del MNIST.  Este conjunto de datos se ha utilizado ampliamente como referencia en la comunidad de aprendizaje automático.

Las imágenes se ven de la siguiente manera.

Hagamos una prueba k-means con 30 centros en este conjunto de datos, y grafiquemos los centros resultantes (recuerde, estos centros están promediando imágenes juntas, por lo que los centros mismos se verán como imágenes razonables).

Sin embargo, una vez más, debemos subrayar que este es un método muy malo para "inferir" el número de cúmulos verdaderos en los datos (que en este caso son 10 imágenes diferentes).  Veamos qué sucede cuando intentamos ejecutar k-means con 10 cúmulos.

El "5" parece faltar en su mayor parte (posiblemente combinado con el "3", y hay una mezcla de "4", "7", "9" caracteres.  Si miráramos la pérdida sobre el número de cúmulos.  Esencialmente, lo que sucedió en nuestro ejemplo con algunos centros contenidos dentro de una sola clase, mientras que otros están dispersos en múltiples clases, es exactamente lo que está sucediendo aquí.  Una mejor inicialización como k-means++ y la ejecución de más iteraciones puede ayudar un poco, pero no arregla el problema fundamental.  Más bien, lo que está sucediendo es simplemente el hecho de que en el espacio de píxeles de estas imágenes, no es necesario que las diferentes imágenes se agrupen como quisiéramos: un "1" podría estar mucho más cerca en distancia de imagen de un "7" que de otro "1".


Si graficamos la pérdida sobre diferentes números de clústers obtenemos:

Definitivamente no seríamos capaces de decir que había 10 clases reales con sólo mirar esta trama.  El mensaje es que si alguien trata de afirmar que ejecutó k-medios (o realmente la mayoría de cualquier algoritmo de agrupación) para determinar que "hay" X número de cúmulos en los datos, uno puede dudar.  